### Part 0b: Merging all the datasets into one and exporting it

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk

In [5]:
### Ingest the data files
cleaned_hpi_price_data = pd.read_csv('data/cleaned data/cleaned_hpi_price_data.csv')
recession_flag = pd.read_csv('data/cleaned data/recession_flag_yearly.csv')
fed_funds_rate = pd.read_csv('data/fedfundsrate.csv')
fed_funds_rate_year = pd.read_csv('data/cleaned data/yearly_fed_rates_mean.csv')
med_hh_income_us = pd.read_csv('data/cleaned data/median_household_income_us.csv')
unemployment_us = pd.read_csv('data/cleaned data/unemployment_in_us.csv')

In [6]:
cleaned_hpi_price_data = cleaned_hpi_price_data.drop(['zip_code', 'normalized_sale_price_2012'], axis = 1)
cleaned_hpi_price_data.head(5)

,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI from 2012,HPI with 2012 base,normalized_sale_price
0,1001.0,1984,.,100.00,279.95,0.357207,58369.688570
1,1001.0,1985,16.00,116.00,279.95,0.414360,67708.838742
2,1001.0,1986,14.21,132.48,279.95,0.473227,77328.163418
3,1001.0,1987,21.08,160.41,279.95,0.572995,93630.817436
4,1001.0,1988,17.63,188.68,279.95,0.673977,110131.928394


In [61]:
recession_flag.head(10)

,YEAR,RECESSION_FLAG
0,1967,0
1,1968,0
2,1969,1
3,1970,1
4,1971,0
5,1972,0
6,1973,1
7,1974,1
8,1975,1
9,1976,0


In [62]:
fed_funds_rate_year.head(10)

,YEAR,avg_rate_for_year
0,1954,1.008333
1,1955,1.785000
2,1956,2.728333
3,1957,3.105000
4,1958,1.572500
5,1959,3.305000
6,1960,3.215833
7,1961,1.955000
8,1962,2.708333
9,1963,3.178333


In [7]:
med_hh_income_us.head()

,DATE,Median_hh_income,Year
0,1984-01-01,55828,1984
1,1985-01-01,56871,1985
2,1986-01-01,58920,1986
3,1987-01-01,59624,1987
4,1988-01-01,60115,1988


In [9]:
unemployment_us.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year_Average
0,1984,8.0,7.8,7.8,7.7,7.4,7.2,7.5,7.5,7.3,7.4,7.2,7.3,7.7
1,1985,7.3,7.2,7.2,7.3,7.2,7.4,7.4,7.1,7.1,7.1,7.0,7.0,7.2
2,1986,6.7,7.2,7.2,7.1,7.2,7.2,7.0,6.9,7.0,7.0,6.9,6.6,6.7
3,1987,6.6,6.6,6.6,6.3,6.3,6.2,6.1,6.0,5.9,6.0,5.8,5.7,6.2
4,1988,5.7,5.7,5.7,5.4,5.6,5.4,5.4,5.6,5.4,5.4,5.3,5.3,5.5


In [21]:
unemployment_us_avg = unemployment_us[['Year', 'Year_Average']]
unemployment_us_avg = unemployment_us_avg.rename(columns={'Year_Average':"Year_Avg_Unempl"})
unemployment_us_avg.head()

,Year,Year_Avg_Unempl
0,1984,7.7
1,1985,7.2
2,1986,6.7
3,1987,6.2
4,1988,5.5


In [25]:
# merge all the data into a single dataframe
total_market_data = cleaned_hpi_price_data.merge(recession_flag, left_on=['Year'], right_on=['YEAR'])
total_market_data = total_market_data.merge(fed_funds_rate_year, left_on=['Year'], right_on=['YEAR'])
total_market_data = total_market_data.merge(med_hh_income_us, left_on=['Year'], right_on = ['Year'])
total_market_data = total_market_data.merge(unemployment_us_avg, left_on = ['Year'], right_on = ['Year'])

In [26]:
total_market_data

,Five-Digit ZIP Code,Year,Annual Change (%),HPI,HPI from 2012,HPI with 2012 base,normalized_sale_price,YEAR_x,RECESSION_FLAG,YEAR_y,avg_rate_for_year,DATE,Median_hh_income,Year_Avg_Unempl
0,1001.0,1984,.,100.00,279.95,0.357207,58369.688570,1984,0,1984,10.225000,1984-01-01,55828,7.7
1,1002.0,1984,15.40,115.40,348.54,0.331095,91121.681896,1984,0,1984,10.225000,1984-01-01,55828,7.7
2,1020.0,1984,14.40,114.40,296.68,0.385601,54643.995979,1984,0,1984,10.225000,1984-01-01,55828,7.7
3,1027.0,1984,24.50,124.50,400.14,0.311141,67051.045930,1984,0,1984,10.225000,1984-01-01,55828,7.7
4,1028.0,1984,15.12,197.78,537.26,0.368127,82988.087351,1984,0,1984,10.225000,1984-01-01,55828,7.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429914,99654.0,2022,14.69,200.49,131.93,1.519670,375838.000000,2022,0,2022,1.683333,2022-01-01,71000,3.8
429915,99701.0,2022,5.15,206.35,154.87,1.332408,232939.000000,2022,0,2022,1.683333,2022-01-01,71000,3.8
429916,99705.0,2022,12.69,205.01,142.38,1.439879,336466.000000,2022,0,2022,1.683333,2022-01-01,71000,3.8
429917,99709.0,2022,9.75,454.21,330.36,1.374894,305912.000000,2022,0,2022,1.683333,2022-01-01,71000,3.8


In [65]:
total_market_data.to_csv('data/cleaned data/total_market_data_merged.csv')